In [4]:
import sys
sys.path.insert(0, 'src')
from read_genome import *

In [35]:
!wget -O resources/excerpt2.fasta https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2024-10-25 08:39:51--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 99.86.66.215, 99.86.66.100, 99.86.66.61, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|99.86.66.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘resources/excerpt2.fasta’

resources/excerpt2. 100%[===================>] 791.12K  1.92MB/s    in 0.4s    

2024-10-25 08:39:53 (1.92 MB/s) - ‘resources/excerpt2.fasta’ saved [810105/810105]



In [56]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

def localAlignment(x, y):
    # Create distance matrix
    m, n = len(x), len(y)
    #fist row with only 0's
    D = [[0]*(n + 1) for _ in range(m + 1)]
  
    # Initialize first column 
    for i in range(m+1):
        D[i][0] = i
 
    # Fill in the rest of the matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            distDiag = D[i-1][j-1] + (x[i-1] != y[j-1])
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the minimum value of the last row
    return min(D[m])

In [61]:
human_genome = readFasta("resources/excerpt2.fasta")

Q1. What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [72]:
t = human_genome
p = 'GCTGATCGATCGTACG'
localAlignment(p, t)

3

Q2. What is the edit distance of the best match between pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1?

In [71]:
t = human_genome
p = 'GATTTACCAGATTGAG'
localAlignment(p, t)

2

Q3. In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match)
between two strings. The function is copied below.

In [78]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

Say we are concerned only with overlaps that (a) are exact matches (no differences allowed),
and (b) are at least K bases long. To make an overlap graph, we could call 
overlap(a,b,min_length=k) on every possible pair of reads from the dataset.
Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read A whose length-6 suffix is GTCCTA.
Say GTCCTA does not occur in any other read in the dataset. In other words, the 6-mer GTCCTA
occurs at the end of read A and nowhere else. It follows that A's suffix cannot possibly overlap
the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read A and a prefix of some other read,
we can ignore any reads that don't contain the length-k suffix of A. This is good news because it can save us
a lot of work!

Here is a suggestion for how to implement this idea. You don't have to do it this way, but this might help you.
Let every k-mer in the dataset have an associated Python Set object, which starts out empty.  We use a Python dictionary
to associate each k-mer with its corresponding Set.

1. For every k-mer in a read, we add the read
to the Set object corresponding to that k-mer. If our read is GATTA and k=3, we would add GATTA to the Set objects
for GAT, ATT and TTA. We do this for every read so that, at the end, each Set contains all reads containing
the corresponding k-mer.
2. Now, for each read A, we find all overlaps involving a suffix of A. To do this, we take A's length-k suffix,
find all reads containing that k-mer (obtained from the corresponding Set) and call overlap(a,b,min_length=k)
for each.

The most important point is that we do not call overlap(a,b,min_length=k) if B does not contain
the length-k suffix of A.

Download and parse the read sequences from the provided Phi-X FASTQ file. We'll just use their base sequences,
so you can ignore read names and base qualities. Also, no two reads in the FASTQ have the same sequence of bases.
This makes things simpler.

Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read
with itself; if a read has a suffix/prefix match to itself, ignore that match. Ignore reverse complements.
* Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset,
and maybe much less than that. (Our solution takes about 3 seconds.) If your function is much slower,
there is a problem somewhere.
* Hint 2: Remember not to overlap a read with itself. If you do, your answers will be too high.
* Hint 3: You can test your implementation by making up small examples, then checking that
(a) your implementation runs quickly, and (b) you get the same answer as if you had simply called
overlap(a,b,min_length=k) on every pair of reads.  We also have provided a couple
[examples you can check against](https://nbviewer.jupyter.org/github/BenLangmead/ads1-hw-examples/blob/master/hw3_overlap_all.ipynb).

In [65]:
!wget -P resources/ https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

--2024-10-25 08:58:14--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 99.86.66.118, 99.86.66.215, 99.86.66.61, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|99.86.66.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘resources/ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  2.97MB/s    in 0.8s    

2024-10-25 08:58:16 (2.97 MB/s) - ‘resources/ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [131]:
from collections import defaultdict
def overlap_all_pairs(reads, k):
    k_dict = defaultdict(set)
    for read in reads:
        for kmer_st in range(len(read)-k + 1):
            k_dict[read[kmer_st:kmer_st + k]].add(read)
    ans = []
    for read in reads:
        for o in k_dict[read[-k:]]:
            if o != read and overlap(read, o, k):
                ans.append((read, o))

    return ans
    

In [132]:
reads= ['ABCDEFG', 'EFGHIJ', 'HIJABC']
overlap_all_pairs(reads, 3)

[('ABCDEFG', 'EFGHIJ'), ('EFGHIJ', 'HIJABC'), ('HIJABC', 'ABCDEFG')]

In [133]:
overlap_all_pairs(reads, 4)

[]

In [134]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
overlap_all_pairs(reads, 4)

[('CGTACG', 'GTACGA'),
 ('CGTACG', 'TACGTA'),
 ('CGTACG', 'TACGAT'),
 ('CGTACG', 'GTACGT'),
 ('TACGTA', 'ACGTAC'),
 ('TACGTA', 'CGTACG'),
 ('GTACGT', 'TACGTA'),
 ('GTACGT', 'ACGTAC'),
 ('ACGTAC', 'GTACGA'),
 ('ACGTAC', 'GTACGT'),
 ('ACGTAC', 'CGTACG'),
 ('GTACGA', 'TACGAT')]

In [135]:
reads, _ = readFastq('resources/ERR266411_1.for_asm.fastq')


In [136]:
overlapped_pairs = overlap_all_pairs(reads, 30)
len(overlapped_pairs)

904746

Q4. Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge?  (In other words, how many reads have a suffix involved in an overlap?)

In [141]:

len({x for x, _ in overlapped_pairs})

7161